In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os.path

In [14]:
import os,shutil

In [15]:
# 원본 데이터셋을 압축 해제한 디렉터리 경로
original_dataset_dir = '/Users/user/Downloads/dogs-vs-cats/train/train'

# 소규모 데이터셋을 저장할 디렉터리
base_dir = '/Users/user/Downloads/dogs-vs-cats/cats_and_dogs_small'
if os.path.exists(base_dir):  # 반복적인 실행을 위해 디렉토리를 삭제합니다.
    shutil.rmtree(base_dir)   # 이 코드는 책에 포함되어 있지 않습니다.
os.mkdir(base_dir)

# 훈련, 검증, 테스트 분할을 위한 디렉터리
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# 훈련용 고양이 사진 디렉터리
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# 훈련용 강아지 사진 디렉터리
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# 검증용 고양이 사진 디렉터리
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# 검증용 강아지 사진 디렉터리
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# 테스트용 고양이 사진 디렉터리
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

# 테스트용 강아지 사진 디렉터리
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

# 처음 1,000개의 고양이 이미지를 train_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

# 다음 500개 고양이 이미지를 validation_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 고양이 이미지를 test_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# 처음 1,000개의 강아지 이미지를 train_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)  # shutil이라는 복사하는 
    
# 다음 500개 강아지 이미지를 validation_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 강아지 이미지를 test_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)


In [16]:
from keras import layers
from keras import models

## 강아지/고양이 분류를 위한 소규모 컨브넷 만들기

In [38]:
model=models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation='relu',
                       input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

## 모델 훈련 설정하기

In [18]:
from keras import optimizers

In [39]:
model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

## ImageDataGenerator를 사용하여 
## 디렉토리에서 이미지 읽기

In [21]:
from keras.preprocessing.image import ImageDataGenerator

In [40]:
train_datagen=ImageDataGenerator(rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255)

train_gen=train_datagen.flow_from_directory(
'/Users/user/Downloads/hand/handwriting_shape/train',
target_size=(150,150),
class_mode='categorical',
    batch_size=20 # batch_size:학습시키는 속도
)

test_gen=test_datagen.flow_from_directory(
'/Users/user/Downloads/hand/handwriting_shape/test',
target_size=(150,150),
class_mode='categorical',
    batch_size=20 # batch_size:학습시키는 속도
)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


### Q
배치사이즈는 어떻게 설정하는가
일반적으로 전체데이터의 몇%씩 학습시킨다 라거나 
통상적으로 정하는 사이즈가 있는지?
아니면 학습속도빠르게 주는게 좋을 것 같은데 
결과에 영향을 많이 주는지도 궁금

### A
결과에 영향 많이 주지 않고,
2의 지수승 사이즈로 통상적으로 결정
오히려 적게 여러번 학습시키는게 빠를 수 있다

In [41]:
history=model.fit_generator(
train_gen,
steps_per_epoch=100,lepochs=30, validation_data=test_gen, validation_steps=50)

Epoch 1/30


ValueError: Error when checking target: expected dense_10 to have shape (1,) but got array with shape (3,)

In [ ]:
import mat